In [1]:
import pandas as pd
import numpy as np

In [2]:
months = [10, 11, 12, 1]

In [3]:
df = pd.read_csv("http://noaa-ghcn-pds.s3.amazonaws.com/csv/by_station/USC00087205.csv")

# Select date, element, and data value columns and select the 1991-2020 period.
df = df[["DATE", "ELEMENT", "DATA_VALUE"]]
df = df[(df["DATE"] >= 19910101) & (df["DATE"] <= 20201231)]
df = df[df["ELEMENT"].isin(["TMIN"])]
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

# Create pivot table from Element column and convert data to degrees F.
pivot = df.pivot(index="DATE", columns="ELEMENT", values="DATA_VALUE").reset_index()
pivot["TMIN"] = (pivot["TMIN"] / 10) * (9 / 5) + 32
pivot_ondj = pivot[pivot["DATE"].dt.month.isin(months)]

pivot_ondj

/var/folders/8t/4dcpfx6j29d858grh_vp_d_m0000gn/T/ipykernel_64236/1658809228.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("http://noaa-ghcn-pds.s3.amazonaws.com/csv/by_station/USC00087205.csv")


ELEMENT,DATE,TMIN
0,1991-01-01,62.06
1,1991-01-02,64.94
2,1991-01-03,64.04
3,1991-01-04,64.04
4,1991-01-05,62.96
...,...,...
10677,2020-12-27,33.98
10678,2020-12-28,48.02
10679,2020-12-29,53.06
10680,2020-12-30,55.04


In [4]:
# Get frost days
frost = pivot[pivot["TMIN"] <= 32]
frost.head()

ELEMENT,DATE,TMIN
46,1991-02-16,30.02
47,1991-02-17,28.94
375,1992-01-17,28.04
379,1992-01-21,32.00
383,1992-01-25,32.00


In [5]:
# Number of frost days for each month.
numFrostDays = frost.groupby(frost["DATE"].dt.month).agg(
    {"TMIN": "count"}, as_index=False
)
numFrostDays

ELEMENT,TMIN
DATE,
1,56
2,19
3,3
9,1
11,1
12,18


In [6]:
numOctFrostDays = 0 / 30
numNovFrostDays = 1 / 30
numDecFrostDays = 18 / 30
numJanFrostDays = 56 / 30

In [7]:
print("Mean number of frost days for October:", numOctFrostDays)
print("Mean number of frost days for November:", numNovFrostDays)
print("Mean number of frost days for December:", numDecFrostDays)
print("Mean number of frost days for January:", numJanFrostDays)

Mean number of frost days for October: 0.0
Mean number of frost days for November: 0.03333333333333333
Mean number of frost days for December: 0.6
Mean number of frost days for January: 1.8666666666666667


In [8]:
# Get freeze days
freeze = pivot[pivot["TMIN"] <= 28]
freeze.head()

ELEMENT,DATE,TMIN
1490,1995-02-09,26.96
1809,1995-12-25,26.06
1810,1995-12-26,26.06
1811,1995-12-27,17.96
1851,1996-02-05,24.98


In [9]:
# Number of frost days for each month.
numFreezeDays = (
    freeze.groupby(freeze["DATE"].dt.month).agg({"TMIN": "count"}).reset_index()
)
numFreezeDays

ELEMENT,DATE,TMIN
0,1,15
1,2,2
2,9,1
3,12,5


In [10]:
numOctFreezeDays = 0 / 30
numNovFreezeDays = 0 / 30
numDecFreezeDays = 5 / 30
numJanFreezeDays = 15 / 30

In [11]:
print("Mean number of freeze days for October:", numOctFreezeDays)
print("Mean number of freeze days for November:", numNovFreezeDays)
print("Mean number of freeze days for December:", numDecFreezeDays)
print("Mean number of freeze days for January:", numJanFreezeDays)

Mean number of freeze days for October: 0.0
Mean number of freeze days for November: 0.0
Mean number of freeze days for December: 0.16666666666666666
Mean number of freeze days for January: 0.5


In [12]:
enso = pd.read_csv(
    "https://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices", sep=r"\s+"
)

enso = enso.rename(
    columns={
        "ANOM": "ANOM1+2",
        "ANOM.1": "ANOM3",
        "ANOM.2": "ANOM4",
        "ANOM.3": "ANOM3.4",
    }
)
enso = enso[(enso["YR"] >= 1991) & (enso["YR"] <= 2020)]

enso

,YR,MON,NINO1+2,ANOM1+2,NINO3,ANOM3,NINO4,ANOM4,NINO3.4,ANOM3.4
108,1991,1,23.73,-0.78,25.63,-0.05,28.62,0.40,26.89,0.33
109,1991,2,25.70,-0.40,26.28,-0.10,28.40,0.30,26.87,0.14
110,1991,3,26.32,-0.28,26.96,-0.20,28.38,0.15,27.16,-0.08
111,1991,4,25.21,-0.52,27.39,-0.19,28.73,0.22,27.89,0.08
112,1991,5,24.65,0.04,27.44,0.22,29.18,0.39,28.13,0.25
...,...,...,...,...,...,...,...,...,...,...
463,2020,8,20.17,-0.69,24.79,-0.43,28.36,-0.34,26.38,-0.52
464,2020,9,19.82,-0.76,24.21,-0.80,28.25,-0.43,26.12,-0.64
465,2020,10,20.12,-0.76,24.19,-0.90,27.97,-0.71,25.64,-1.13
466,2020,11,20.95,-0.67,24.17,-1.03,27.98,-0.70,25.59,-1.23


In [13]:
pivot["ym"] = pivot["DATE"].dt.strftime("%Y-%m")

freezeDaysPerMon = pivot[pivot["TMIN"] < 28].groupby(pivot["ym"]).agg({"TMIN": "count"})

freezeDaysPerMon 

ELEMENT,TMIN
ym,
1995-02,1
1995-12,3
1996-02,1
1997-01,1
1999-01,3
2001-01,1
2003-01,1
2004-09,1
2008-01,1


In [14]:
print(enso["ANOM1+2"].corr(freezeDaysPerMon["TMIN"]))
print(enso["ANOM3"].corr(freezeDaysPerMon["TMIN"]))
print(enso["ANOM4"].corr(freezeDaysPerMon["TMIN"]))
print(enso["ANOM3.4"].corr(freezeDaysPerMon["TMIN"]))

nan
nan
nan
nan
